### Домашняя работа к Уроку 9 (II часть. Запросы на сервер)
### Студент: Абрамов А.В.

In [21]:
import requests
import json 
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse
import time

In [10]:
# Пример данных
data = (26.7, 7.7, 200)

In [11]:
# формируем запрос
def send_json(x):
    bmi, HbA1c_level, blood_glucose_level = x
    #print(bmi, HbA1c_level, blood_glucose_level)
    body = {
        'bmi': bmi, 
        'HbA1c_level': HbA1c_level,
        'blood_glucose_level': blood_glucose_level
        }
    #myurl = 'http://948a-35-230-58-89.ngrok.io/' + '/predict'
    myurl = 'http://127.0.0.1:8080/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [12]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.9243304034342975


In [14]:
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [20]:
# Проверим массовый запрос
N=20
predictions = X_test[['bmi', 'HbA1c_level', 'blood_glucose_level']].iloc[:N].apply(lambda x: send_json(x), axis=1)
predictions

0     0.005831
1     0.001685
2     0.000465
3     0.008601
4     0.550291
5     0.345887
6     0.004812
7     0.002988
8     0.051806
9     0.008250
10    0.002911
11    0.010641
12    0.000101
13    0.181958
14    0.001166
15    0.046976
16    0.000244
17    0.000018
18    0.482937
19    0.000169
dtype: float64

In [22]:
precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.5502914968016707, F-Score=0.667, Precision=1.000, Recall=0.500


In [23]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.8888888888888888

#### Вывод: всё работает.